**This notebook calculate the nERMC using the S&Z's method for simulating RGCs**

In [2]:
import numpy as np
import statistics as ST
import pandas as pd
import scipy.stats
import random
import copy
import time
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib import colors

##  **0. Functions required**

In [51]:
def new_property_matrix_generator(list_order,property_dictionary):
# This function will take a list of short AA names (represent the permutated UGC table) and property dictionary to generate a AA property matrix
    temp_matrix=[]
    for aa_short_name in list_order:
        temp_matrix.append(property_dictionary[aa_short_name])
    return(np.array(temp_matrix))

In [52]:
# This function will do the point mutation for given sequence and depend on the given position
def point_mutation(input_codon,position):
    ref_set={'A','C','G','T'}-{input_codon[position-1]}
    o_list=[]
    if position==1:
        for x in ref_set:
            o_list.append(x+input_codon[1:])
    if position==2:
        for x in ref_set:
            o_list.append(input_codon[0]+x+input_codon[2])
    if position==3:
        for x in ref_set:
            o_list.append(input_codon[:2]+x)
    return(o_list)

In [53]:
def Ts_Tv_position_matrix_generator(mutation_pairs,alpha,beta):
# This function with take a list of mutation pairs with length L, the transition to transversion bias alpha,
# and the position bias beta to generate a LxL matrix with only diagnal element
    Ts_Tv_dic=[set(['A','G']),set(['T','C'])]# This list include all the position transition pair
    weight_vector=[]
    for x in mutation_pairs:
        temp_counter=1
        for a,b in zip(x[0],x[1]):# loop over two codons in the pair and find the mistmatch position and nucleotide
            if a!=b:
                temp1=a
                temp2=b
                break
            else:
                temp_counter+=1
        temp_set={temp1,temp2}
        if temp_set in Ts_Tv_dic:
            weight=alpha[temp_counter]*beta[temp_counter]
        else:
            weight=1*beta[temp_counter]
        weight_vector.append(weight)
    return( np.diag(weight_vector))

In [54]:
def mutation_matrix_generator_science(mutation_pairs):
# This function take into a list of mutation pairs, generate a matrix according to the UGC_table
    codon_to_order={}
    for x ,y in zip(UGC_table.keys(),range(64)):
        codon_to_order[x]=y
    temp_matrix=[]
    for x in mutation_pairs:
        p1=codon_to_order[x[0]]
        p2=codon_to_order[x[1]]
        temp_vector=[0] * 64
        if p1!=p2:
            temp_vector[p1]=1
            temp_vector[p2]=-1
        temp_matrix.append(temp_vector)
    return(np.array(temp_matrix))

In [55]:
def block_matrix_generator_science(new_block_order,aa_list):
# This function take into a list of new block order and aa_list order
    block_matrix=[]
    for x in new_block_order:
        temp_vector=[0] * 21
        # this is find each codon correspond to which AA
        temp=aa_list.index(x)
        temp_vector[temp]=1
        block_matrix.append(temp_vector)
    return(np.array(block_matrix))

In [56]:
def ERMC_calculation_ultimate_science(weight_matrix,mutation_matrix,block_matrix,pp_matrix,total_weight,version):
# The first input is a matrix with all the weight information, the second matrix contains the information of codon mutation, the thrid matrix contains the information for aa properties,
# total_weight is used for calculating the average, version =1 will lead to the calcualtion that only consider positive cost, while version=2 will consider both positive and negative cost
    # because my mutation matrix give wt codon coefficient of 1 while mutated codon coefficient of -1. I need to use the negative value    
    mutation_matrix=-mutation_matrix
    temp=mutation_matrix.dot(block_matrix.dot(pp_matrix))    
    if version==1:
        # only consider the postive cost
        temp[temp<0]=0
    return(np.sum(weight_matrix.dot(temp),axis=0)/total_weight)

In [57]:
def frequency_weight_matrix_generator(mutation_pairs,frequency_dictionary):
# mutation_paris store all possible pairs of mutations.
# frequency_dictionary store frequency value for all codon
    freq_list=[]
    for x in mutation_pairs:
        freq=frequency_dictionary[x[0]] # get the frequency according to the original codon
        freq_list.append(freq)
    return(np.diag(freq_list)/sum(freq_list))

In [58]:
def read_benchmark(input_address):
    temp_list=[]
    with open(input_address,'r') as handler:
        handler.readline()
        temp=handler.readline().strip().split(',')
        temp_list=[float(x) for x in temp]
    return(temp_list)

In [59]:
def read_real_data(input_address): # read ERMC data to store in a matrix
    temp_matrix=[]
    with open(input_address,'r') as handler:
        handler.readline()
        temp=handler.readline().strip()
        while temp:
            temp=handler.readline().strip()
            temp_matrix.append([float(x) for x in temp.split(',')])
            temp=handler.readline().strip()
    return(np.array(temp_matrix))

In [60]:
def ERMC_to_Rank_modified(ERMC_array,UGC_benchmark,header_name): # convert ERMC array to a rank array and convert the ERMC benchmark to rank
# ERMC_array each row is a random codon, each column is a property
    Rank_array=np.argsort(np.argsort(ERMC_array, axis=0),axis=0)+1
    k=ERMC_array<UGC_benchmark
    UGC_rank=np.sum(k,axis=0)
    temp=np.sum(k[:,[0,2]],axis=1)
    UGC_NC_rank=sum(temp==2)# This step is to find random genetic code that are better than UGC in both carbon and nitrogen
    UGC_rank=np.append(UGC_rank,UGC_NC_rank)
    UGC_output=header_name+"_UGC"
    file_b=open(UGC_output,'w')
    stringtowrite1="{}\n".format(','.join(['Carbon_number', 'Oxygen_number', 'Nitrogen_number','NC']))
    file_b.write(stringtowrite1)
    stringtowrite1="{}\n".format(','.join([str(x) for x in UGC_rank]))
    file_b.write(stringtowrite1)
    return(UGC_rank)
    rank_output=header_name
    file_a=open(rank_output,'w')
    stringtowrite1="{}\n".format(','.join(p_name_list))
    file_a.write(stringtowrite1) 
    for x in Rank_array:
        stringtowrite="{}\n".format(','.join([str(i) for i in x]))
        file_a.write(stringtowrite) 
    

##  **1 Preparation**

###  **1.1 Input**

In [3]:
# This is the absolute path
Path_ab2="Data/"
#  This is the aminoacid properties
aa2=Path_ab2+'Nutrient_metrics.csv'
Path_ab3="Output/"

In [4]:
# nutrient data
Nutrient_data=pd.read_csv(aa2)
Nutrient_data.set_index('Amino acid',inplace=True)
p_name_list=list(Nutrient_data.columns)
#  random genetic code
simulated_lib=Path_ab2+"New_RGC_1e6"
#  codon frequency data
codon_freq_address=Path_ab2+'ModelOrganisms.csv'

In [5]:
# read the codon frequency information into a pandas dataframe
codon_freq=pd.read_csv(codon_freq_address)
# Drop unnecessary columns
codon_freq=codon_freq.drop(columns=['Unnamed: 0','Assembly','Translation Table','Organelle','Division'])

In [6]:
# this list contains the codon names
codon_freq_name_list=codon_freq.columns[8:]

###  **1.2 UGC structure**

In [31]:
# make a dictionary, the codon is the key, while the aa one leter name is the value
UGC_table = { 
'TTT':'F', 'TTC':'F', 'TTA':'L', 'TTG':'L', 
'TCT':'S', 'TCC':'S', 'TCA':'S', 'TCG':'S', 
'TAT':'Y', 'TAC':'Y', 'TAA':'_', 'TAG':'_', 
'TGT':'C', 'TGC':'C', 'TGA':'_', 'TGG':'W', 
'CTT':'L', 'CTC':'L', 'CTA':'L', 'CTG':'L', 
'CCT':'P', 'CCC':'P', 'CCA':'P', 'CCG':'P', 
'CAT':'H', 'CAC':'H', 'CAA':'Q', 'CAG':'Q', 
'CGT':'R', 'CGC':'R', 'CGA':'R', 'CGG':'R',     
'ATT':'I', 'ATC':'I', 'ATA':'I', 'ATG':'M', 
'ACT':'T', 'ACC':'T', 'ACA':'T', 'ACG':'T', 
'AAT':'N', 'AAC':'N', 'AAA':'K', 'AAG':'K', 
'AGT':'S', 'AGC':'S', 'AGA':'R', 'AGG':'R',                  
'GTT':'V', 'GTC':'V', 'GTA':'V', 'GTG':'V',  
'GCT':'A', 'GCC':'A', 'GCA':'A', 'GCG':'A',
'GAT':'D', 'GAC':'D', 'GAA':'E', 'GAG':'E', 
'GGT':'G', 'GGC':'G', 'GGA':'G', 'GGG':'G', 
       
    } 

In [29]:
# Here I create a dictionary where the keys is the 4 element square block name. 
# The name is the order number from left to right. And from top to bottom.
# '_' stand for the stop codon
UGC_block_information={
    '0':['F','F','L','L'],'1':['S','S','S','S'],'2':['Y','Y','_','_'],'3':['C','C','_','W'],'4':['L','L','L','L'],
    '5':['P','P','P','P'],'6':['H','H','Q','Q'],'7':['R','R','R','R'],'8':['I','I','I','M'],'9':['T','T','T','T'],
    '10':['N','N','K','K'],'11':['S','S','R','R'],'12':['V','V','V','V'],'13':['A','A','A','A'],'14':['D','D','E','E'],
    '15':['G','G','G','G'],
}

In [30]:
# make a dictionary, the block name as the key, a list of condon correponding to this block is the value
UGC_block={}
for key,value in UGC_table.items():
    if value in UGC_block.keys():
        UGC_block[value].append(key)
    else:
        UGC_block[value]=[key]

###  **1.3 Seperate species accoridng to kingdom**

In [20]:
Bacteria_group=[2097,1423,562,354,100226]
Extremophiles_group=[41673,272844,1917166,694429,3046,797304,243230]
Unicelluar_Eukaryotes_group=[4896,4932,35128,162425,2903,3055]
Plants_group=[3702,34305,3694,3218,4530,4577]
Animals_group=[6239,8364,7070,7955,9031,9598,10181,10090,10116,8296,9615,9606,7227,9685,59729]

In [21]:
Total_frequency_table=[Bacteria_group,Extremophiles_group,Unicelluar_Eukaryotes_group,Plants_group,Animals_group]
Group_name_list=['Bacteria','Extremophiles','Unicelluar_Eukaryotes','Plants','Animals']

##  **2 Calculation**

###  **2.1 ERMC calculation**

In [9]:
# input data list, the 64 codons, include the stop codon
initial_list=[]
for key,value in UGC_table.items():
    initial_list.append(key)

In [15]:
# Here I convert the property dataframe into a dic, the key is the one letter name for AA, and values are corresponding property value
property_dic={}
for index, row in Nutrient_data.iterrows():
    property_dic[index]=[x for x in row] # The aa short name is a key and the propery is the value
# adding the option of stop codon.
property_dic['_']=[0,0,0]
amino_acid_list=list(property_dic.keys())
# property matrix generation
UGC_pp_matrix=new_property_matrix_generator(property_dic.keys(),property_dic)

In [21]:
# Generate a dict, the key is the original codon and the value is point mutated codon
mutated_dic={}
for x in initial_list:
    l2=[] # substituted codon
    for j in range(1,4):
        l2.append([y for y in point_mutation(x,j)])
    mutated_dic[x]=l2     

#  list of all the mutation pairs
sub_pair_p1=[] # p1 substitution
sub_pair_p2=[] # p2 substitution
sub_pair_p3=[] # p3 substitution
sub_pair_all=[]
for key,value in mutated_dic.items():
    for x in value[0]:
        sub_pair_p1.append([key,x])
    for x in value[1]:
        sub_pair_p2.append([key,x])
    for x in value[2]:
        sub_pair_p3.append([key,x])
sub_pair_all=sub_pair_p1+sub_pair_p2+sub_pair_p3
# output value

In [25]:
# mutation matrics
tt_sub_all=mutation_matrix_generator_science(sub_pair_all)

In [32]:
# define a new list called block order
block_order=list(UGC_block_information.keys())

In [33]:
new_order=[]
for x in block_order:
    new_order+=UGC_block_information[x]

In [38]:
new_block_matrix=block_matrix_generator_science(new_order,amino_acid_list)

In [43]:
# ratio list for Ts/Tv bias
ratio_list=[0.2,1/4,1/3,0.5,1,2,3,4,5]
# I create a list to store dictionarys for Ts/Tv metrics
TsTv_list=[]
TsTv_list.append({1:2,2:5,3:1})
Position_list=[]
Position_list.append({1:0.5,2:0.1,3:1})
for x in range(1,10):
    temp_dict={1:ratio_list[x-1],2:ratio_list[x-1],3:ratio_list[x-1]}
    TsTv_list.append(temp_dict)
    Position_list.append({1:1,2:1,3:1})

In [30]:
import os

###  **2.2 Output individual file**

In [31]:
####  **1.2.1 nERMC output**
Total_pvalue_dic={}
for num,species_group in enumerate(Group_name_list):
    temp_suffix=species_group
    temp_group=Total_frequency_table[num]
    temp_pvalue_dic={}
    General_path=Path_ab3+'1e6_Figure4_NC/'+temp_suffix+'/'
    if not os.path.isdir(General_path):
        os.mkdir(General_path)
    for species_taxid in temp_group:
        species_name=codon_freq[codon_freq['Taxid']==species_taxid].iloc[0,]['Species']
        UGC_rank_summary=[]
        # this is for generating frequency matrix
        freq_value=codon_freq[codon_freq['Taxid']==species_taxid].iloc[0, 8:]
        temp_freq_dic={}
        for name,value in zip(codon_freq_name_list,freq_value):
            temp_freq_dic[name]=value
        sub_general_path=General_path+species_name+'/'
        if not os.path.isdir(sub_general_path):
            os.mkdir(sub_general_path)
        for idx,item in enumerate(TsTv_list):
            sv_list=item
            po_list=Position_list[idx]
            name_suffix='_set'+str(idx)
            # 
            #  UGC
            UGC_tsub_ERMC_address=sub_general_path+'UGC_tsub_ERMC'+name_suffix
            # As for now I only have one output category
            UGC_output_summary=[UGC_tsub_ERMC_address]

            # 1e6 random genetic code        
            tsub_ERMC_address=sub_general_path+'1e6_tsub_ERMC'+name_suffix
            output_summary=[tsub_ERMC_address]

            ####  **1.2.1 Rank and Pvalue output**
            Output_tsub_address=sub_general_path+'1e6_tsub_Rank'+name_suffix


            # This generate the weight matrix that consider both position effect and Ts/Tv bias
            weight_matrix_tsub_all=np.diag([1]*len(sub_pair_all)).dot(Ts_Tv_position_matrix_generator(sub_pair_all,sv_list,po_list))

            temp_freq_matrix=frequency_weight_matrix_generator(sub_pair_all,temp_freq_dic)
            weight_matrix_tsub_all=weight_matrix_tsub_all.dot(temp_freq_matrix)
            trace_tsub_all=weight_matrix_tsub_all.trace()

            ERMC_tsub=ERMC_calculation_ultimate_science(weight_matrix_tsub_all,tt_sub_all,new_block_matrix,UGC_pp_matrix,trace_tsub_all,2)

            ERMC_summary=[ERMC_tsub]

            for x,y in zip(UGC_output_summary,ERMC_summary):
                file_a=open(x,'w')
                file_a.write(','.join([x for x in Nutrient_data.columns]))
                file_a.write('\n')
                stringtowrite="{}\n".format(','.join([str(i) for i in y]))
                file_a.write(stringtowrite)
                file_a.close()



            # This is output file for random code 
            file_list=[]
            for x in output_summary:
                file_list.append(open(x,'w'))
            for x in file_list:
                x.write(','.join([x for x in Nutrient_data.columns]))
                x.write('\n')

            with open(simulated_lib,'r')as handler:
                temp=handler.readline().strip()
                while temp:
                    s_list=temp.split(',') 
                    # Here I new order into a new block matrix
                    new_order=[]
                    for x in s_list:
                        new_order+=UGC_block_information[x]
                    temp_new_block_matrix=block_matrix_generator_science(new_order,amino_acid_list)
                    ERMC_tsub=ERMC_calculation_ultimate_science(weight_matrix_tsub_all,tt_sub_all,temp_new_block_matrix,UGC_pp_matrix,trace_tsub_all,2)
                    ERMC_summary=[ERMC_tsub]
                    for x,y in zip(file_list,ERMC_summary):                           
                        x.write(temp.replace(',', '')+'\n')
                        stringtowrite="{}\n".format(','.join([str(i) for i in y]))
                        x.write(stringtowrite)                                       
                    temp=handler.readline().strip()

            for x in file_list:
                x.close()
            # read UGC data for ERMC
            UGC_tsub_ERMC=read_benchmark(UGC_tsub_ERMC_address)
            # read random genetic code data for ERMC
            tsub_ERMC=read_real_data(tsub_ERMC_address)
            UGC_rank_summary.append(ERMC_to_Rank_modified(tsub_ERMC,UGC_tsub_ERMC,Output_tsub_address)/1e6)
        temp_pvalue_dic[species_name]=UGC_rank_summary
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    Total_pvalue_dic[species_group]=temp_pvalue_dic

21:30:20
04:51:15
10:53:20
17:36:29
09:51:38


###  **2.3 Output pvalue into json file**

In [97]:
import json

In [98]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
json_dump = json.dumps(Total_pvalue_dic, cls=NumpyEncoder)

In [99]:
pvalue_dic_address=Path_ab2+'1e6_NC_pvalue_dictionary.json'
with open(pvalue_dic_address, 'w') as f:
    json.dump(json_dump, f)